In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import glob
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout , Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from sklearn.utils import shuffle
# Any results you write to the current directory are saved as output.

In [ ]:
X_para = []
Y_para = []
myfiles = glob.glob("../input/cell_images/cell_images/Parasitized/*.png")
for file in myfiles:
    kernel = np.array([[0,-1,0],[-1,6,-1],[0,-1,0]])
    img = cv2.filter2D( cv2.resize(cv2.imread(file) , (120,120)) , -1 , kernel)
    image_yuv = cv2.cvtColor(img ,cv2.COLOR_BGR2YUV )
    image_yuv[: ,: , 0] = cv2.equalizeHist(image_yuv[:,:,0])
    image = cv2.cvtColor(image_yuv , cv2.COLOR_YUV2RGB)
    X_para.append(image)
    Y_para.append(1)
    


In [ ]:
X_un , Y_un = [],[]
unfiles = glob.glob("../input/cell_images/cell_images/Uninfected/*.png")
for file in unfiles:
    kernel = np.array([[0,-1,0],[-1,7,-1],[0,-1,0]])
    img = cv2.filter2D( cv2.resize(cv2.imread(file) , (120,120)) , -1 , kernel)
    X_un.append(img)
    Y_un.append(0)

In [ ]:
X = X_para + X_un
Y = Y_para + Y_un
X,Y = shuffle = (X,Y)
X,Y = shuffle = (X,Y)
X,Y = shuffle = (X,Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3 , random_state =42)
X = np.array(X)


## Image Size

In [ ]:
image_size=[120,120,3]

# CNN

In [ ]:
inp = Input(shape = (120 , 120 , 3))
x = Conv2D(filters = 16 , kernel_size = (3,3) , strides = (1,1) , padding = "valid" , kernel_initializer=glorot_uniform(seed = 2))(inp)
x = Activation("relu")(x)
x = Dropout(0.2)(x)
x = Conv2D(filters = 32 , kernel_size = (4,4) , strides = (2,2) , padding = "valid" , kernel_initializer=glorot_uniform(seed = 2))(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size = (2,2) , strides = (2,2) , padding = "valid")(x)
x = Dropout(0.2)(x)
x = Conv2D(filters = 64 , kernel_size = (3,3) , strides = (2,2) , padding = "valid" , kernel_initializer = glorot_uniform(seed = 2))(x)
x = Activation("relu")(x)
x = Dropout(0.2)(x)
x = Conv2D(filters = 128 , kernel_size = (3,3) , strides = (1,1) , padding = "valid" , kernel_initializer = glorot_uniform())(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size = (2,2) , strides = (2,2) , padding = "valid")(x)
x = Dropout(0.2)(x)
x = Conv2D(filters = 256 , kernel_size = (2,2) , strides = (2,2) , padding = "valid" , kernel_initializer = glorot_uniform())(x)
x = Activation("relu")(x)
x = AveragePooling2D(pool_size = (3,3) , strides = (1,1) , padding = "valid")(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(120)(x)
x = Activation("relu")(x)
x = Dropout(0.2)(x)
x = Dense(60)(x)
x = Activation("relu")(x)
x = Dropout(0.2)(x)
x = Dense(10)(x)
x = Activation("relu")(x)
x = Dropout(0.2)(x)
x = Dense(1)(x)
output = Activation("sigmoid")(x)
model  = Model(inputs =inp , outputs = output )

In [ ]:
model.compile(loss = "binary_crossentropy" , optimizer = "adam" , metrics = ["accuracy"])

model.summary()

In [ ]:
history = model.fit(np.array(X_train) ,np.array(Y_train) , epochs = 13 ,validation_split = 0.2 )

In [ ]:
# gives output in 2d array
y_pre = model.predict(np.array(X_test))
# converting 2d array into 1d arrray
y_pre = np.reshape(y_pre ,(8268,) )
print(y_pre)

Y_test = np.array(Y_test)
fil = y_pre > 0.8
y_pre[fil] = 1
fil = y_pre < 0.8
y_pre[fil] = 0

# printing accuracy of th test dataset
np.sum(Y_test == y_pre)/8268

In [ ]:
# classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(Y_test,y_pre))
print(accuracy_score(Y_test,y_pre))

In [ ]:
# plotting Accuracy plot of training and validation accuracy
plt.plot(history.history['acc'])
plt.plot(history.history["val_acc"])
plt.title("Model Accuracy")
plt.xlabel("Accuracy")
plt.ylabel("Epochs")
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
# plotting loss of training and validation dataset
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# VGG 19

In [ ]:
# importing the pre_trained model
pre_trained_model = tf.keras.applications.VGG19(input_shape=(120,120,3),include_top=False,weights='imagenet')

# we want to keep some weights as it is so let's lock above layers of the Network
for layer in pre_trained_model.layers:
    layer.trainable = False
    
pre_trained_model.summary()

x = Flatten()(pre_trained_model.output)

x = Dense(1,activation='sigmoid')(x)

VGG19_model = Model(pre_trained_model.input,x)

VGG19_model.compile(optimizer = 'adam',loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

history = VGG19_model.fit(np.array(X_train) ,np.array(Y_train) , epochs = 10 ,validation_split = 0.2)

In [ ]:
# gives output in 2d array
y_pre = VGG19_model.predict(np.array(X_test))
# converting 2d array into 1d arrray
y_pre_vgg19 = np.reshape(y_pre ,(8268,) )
print(y_pre_vgg19)

Y_test = np.array(Y_test)
fil = y_pre_vgg19 > 0.8
y_pre_vgg19[fil] = 1
fil = y_pre_vgg19 < 0.8
y_pre_vgg19[fil] = 0

# printing accuracy of th test dataset
np.sum(Y_test == y_pre_vgg19)/8268

In [ ]:
# classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(Y_test,y_pre_vgg19))
print(accuracy_score(Y_test,y_pre_vgg19))

In [ ]:
# plotting Accuracy plot of training and validation accuracy
plt.plot(history.history['acc'])
plt.plot(history.history["val_acc"])
plt.title("Model Accuracy")
plt.xlabel("Accuracy")
plt.ylabel("Epochs")
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
# plotting loss of training and validation dataset
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# VGG 16

In [ ]:
# importing the pre_trained model
pre_trained_model = tf.keras.applications.VGG16(input_shape=(120,120,3),include_top=False,weights='imagenet')

# we want to keep some weights as it is so let's lock above layers of the Network
for layer in pre_trained_model.layers:
    layer.trainable = False
    
pre_trained_model.summary()

x = Flatten()(pre_trained_model.output)

x = Dense(1,activation='sigmoid')(x)

VGG16_model = Model(pre_trained_model.input,x)

VGG16_model.compile(optimizer = 'adam',loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

history = VGG16_model.fit(np.array(X_train) ,np.array(Y_train) , epochs = 10 ,validation_split = 0.2)

In [ ]:
# gives output in 2d array
y_pre = VGG16_model.predict(np.array(X_test))
# converting 2d array into 1d arrray
y_pre_vgg16 = np.reshape(y_pre ,(8268,) )
print(y_pre_vgg16)

Y_test = np.array(Y_test)
fil = y_pre_vgg16 > 0.8
y_pre_vgg16[fil] = 1
fil = y_pre_vgg16 < 0.8
y_pre_vgg16[fil] = 0

# printing accuracy of th test dataset
np.sum(Y_test == y_pre_vgg16)/8268

In [ ]:
# classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(Y_test,y_pre_vgg16))
print(accuracy_score(Y_test,y_pre_vgg16))

In [ ]:
# plotting Accuracy plot of training and validation accuracy
plt.plot(history.history['acc'])
plt.plot(history.history["val_acc"])
plt.title("Model Accuracy")
plt.xlabel("Accuracy")
plt.ylabel("Epochs")
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
# plotting loss of training and validation dataset
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# DenseNet 121

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Flatten, Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint   
from tensorflow.keras.utils import to_categorical


Y_train = to_categorical(Y_train)

Y_test = to_categorical(Y_test)

conv_base = DenseNet121(weights="imagenet", include_top = False, input_shape=(120,120,3))

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.summary()

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit(
    np.array(X_train),np.array(Y_train),
    batch_size=64,
    epochs=10,
    callbacks=[checkpoint],
    validation_split=0.2,
    verbose=1
)

In [ ]:
# gives output in 2d array
y_pre_dense = model.predict(np.array(X_test))
# converting 2d array into 1d arrray

Y_test = np.array(Y_test)
fil = y_pre_dense > 0.8
y_pre_dense[fil] = 1
fil = y_pre_dense < 0.8
y_pre_dense[fil] = 0

# printing accuracy of th test dataset
np.sum(Y_test == y_pre_dense)/8268

In [ ]:
# classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(Y_test,y_pre_dense))
print(accuracy_score(Y_test,y_pre_dense))

In [ ]:
# plotting Accuracy plot of training and validation accuracy
plt.plot(history.history['acc'])
plt.plot(history.history["val_acc"])
plt.title("Model Accuracy")
plt.xlabel("Accuracy")
plt.ylabel("Epochs")
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
# plotting loss of training and validation dataset
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# Embedding

In [ ]:
y_pre_dense = y_pre_dense[:,1]


In [ ]:
Y_test = Y_test[:,1]


In [ ]:
print(y_pre)

In [ ]:
y_final = []
for i in range(len(y_pre_dense)):
    if(y_pre_vgg16[i] == y_pre[i]):
        y_final.append(y_pre_vgg16[i])
    elif(y_pre_vgg16[i] == y_pre_dense[i]):
        y_final.append(y_pre_vgg16[i])
    elif(y_pre[i] == y_pre_dense[i]):
        y_final.append(y_pre_vgg19[i])


In [ ]:
y_final = np.array(y_final)


In [ ]:
# classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print(classification_report(Y_test,y_final))
print(accuracy_score(Y_test,y_final))